<a href="https://colab.research.google.com/github/AjManoj/LearningAICollabs/blob/master/Video%20AI%20%3AConsistent_Video_Depth_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [SIGGRAPH 2020] Consistent Video Depth Estimation
[Project website](https://github.com/facebookresearch/consistent_depth)

## Download Code

In [ ]:
%cd /content/
!git clone https://github.com/facebookresearch/consistent_depth.git
%cd consistent_depth
!git submodule update --init --recursive

/content
Cloning into 'consistent_depth'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 57 (delta 0), reused 52 (delta 0), pack-reused 5
Unpacking objects: 100% (57/57), done.
/content/consistent_depth
Submodule 'monodepth/mannequin_challenge' (https://github.com/roxanneluo/mannequinchallenge.git) registered for path 'monodepth/mannequin_challenge'
Submodule 'monodepth/midas_v2' (https://github.com/roxanneluo/MiDaS-1.git) registered for path 'monodepth/midas_v2'
Submodule 'monodepth/monodepth2' (https://github.com/roxanneluo/monodepth2.git) registered for path 'monodepth/monodepth2'
Submodule 'third_party/OpticalFlowToolkit' (https://github.com/roxanneluo/OpticalFlowToolkit.git) registered for path 'third_party/OpticalFlowToolkit'
Submodule 'third_party/colmap' (https://github.com/colmap/colmap.git) registered for path 'third_party/colmap'
Submodule 'flownet2' (https://github.com/rox

## Prepare Environment

### Download model files

In [ ]:
!./scripts/download_model.sh

allexport      	off
braceexpand    	on
emacs          	off
errexit        	on
errtrace       	off
functrace      	off
hashall        	on
histexpand     	off
history        	off
ignoreeof      	off
interactive-comments	on
keyword        	off
monitor        	off
noclobber      	off
noexec         	off
noglob         	off
nolog          	off
notify         	off
nounset        	off
onecmd         	off
physical       	off
pipefail       	off
posix          	off
privileged     	off
verbose        	off
vi             	off
xtrace         	on
++ mkdir -p checkpoints
++ gdown 'https://drive.google.com/uc?id=1hF8vS6YeHkx3j2pfCeQqqZGwA_PJq_Da' -O checkpoints/flownet2.pth
Downloading...
From: https://drive.google.com/uc?id=1hF8vS6YeHkx3j2pfCeQqqZGwA_PJq_Da
To: /content/consistent_depth/checkpoints/flownet2.pth
650MB [00:05, 126MB/s]


### Install Python Dependencies

In [ ]:
!./scripts/install.sh

Streaming output truncated to the last 5000 lines.
                                   ^
/usr/local/lib/python3.7/dist-packages/torch/include/ATen/core/interned_strings.h:569:1: note: in expansion of macro ‘FORALL_NS_SYMBOLS’
 FORALL_NS_SYMBOLS(DEFINE_SYMBOL)
 ^~~~~~~~~~~~~~~~~
/usr/local/lib/python3.7/dist-packages/torch/include/ATen/core/interned_strings.h:97:11: note:   ‘c10::aten::str’
   _(aten, str)                       \
           ^
/usr/local/lib/python3.7/dist-packages/torch/include/ATen/core/interned_strings.h:568:35: note: in definition of macro ‘DEFINE_SYMBOL’
   namespace ns { constexpr Symbol s(static_cast<unique_t>(_keys::ns##_##s)); }
                                   ^
/usr/local/lib/python3.7/dist-packages/torch/include/ATen/core/interned_strings.h:569:1: note: in expansion of macro ‘FORALL_NS_SYMBOLS’
 FORALL_NS_SYMBOLS(DEFINE_SYMBOL)
 ^~~~~~~~~~~~~~~~~
In file included from /usr/local/lib/python3.7/dist-packages/torch/include/c10/core/Device.h:5:0,
               

### [Optional] Install COLMAP

> **COLMAP** is not required for running the demo sequence. But if you want to run on your own video and let **COLMAP** to estimate camera pose, please install **COLMAP**. Installation takes a couple minutes.




In [ ]:
# Install packages
!sudo apt-get install \
    git \
    cmake \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-regex-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libfreeimage-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libcgal-qt5-dev

# download precompiled colmap packages
%pushd /content/
!wget https://www.dropbox.com/s/zvbw6h8nrlm3hc3/colmap-packages.zip?dl=1 -O colmap-packages.zip
!unzip colmap-packages.zip -d colmap-packages
# ceres-solver make install
%cd colmap-packages/
%pushd ceres-solver/build/
!sudo make install
%popd
# colmap make install
%pushd colmap/build
!sudo make install
!CC=/usr/bin/gcc-6 CXX=/usr/bin/g++-6 cmake ..
%popd
%popd  # pop /content

# test colmap
!colmap
!which colmap

Streaming output truncated to the last 5000 lines.
  inflating: colmap-packages/content/colmap-packages/colmap/lib/VLFeat/gmm.h  
  inflating: colmap-packages/content/colmap-packages/colmap/lib/VLFeat/liop.h  
  inflating: colmap-packages/content/colmap-packages/colmap/lib/VLFeat/sift.c  
  inflating: colmap-packages/content/colmap-packages/colmap/lib/VLFeat/shuffle-def.h  
  inflating: colmap-packages/content/colmap-packages/colmap/lib/VLFeat/mser.c  
  inflating: colmap-packages/content/colmap-packages/colmap/lib/VLFeat/ikmeans_init.tc  
  inflating: colmap-packages/content/colmap-packages/colmap/lib/VLFeat/mathop_avx.h  
  inflating: colmap-packages/content/colmap-packages/colmap/lib/VLFeat/homkermap.c  
  inflating: colmap-packages/content/colmap-packages/colmap/lib/VLFeat/imopv.h  
  inflating: colmap-packages/content/colmap-packages/colmap/lib/VLFeat/imopv_sse2.h  
  inflating: colmap-packages/content/colmap-packages/colmap/lib/VLFeat/gmm.c  
  inflating: colmap-packages/content/

## Prepare Input Video
You can either try out the demo we provied (no need to install COLMAP) or run on your own video sequence (COLMAP required).

### Try out Our Demo
Download the demo video together with its precomputed COLMAP results

In [ ]:
%%capture
!./scripts/download_demo.sh results/ayush
path = 'results/ayush'
video_file = 'data/videos/ayush.mp4'
camera_params = "1671.770118, 540, 960" 
camera_model = "SIMPLE_PINHOLE"

### Use Your Own Video
- [Optional] **Calibrate and set camera parameters** using [`PINHOLE` (fx, fy, cx, cy) or `SIMPLE_PINHOLE` (f, cx, cy) model](https://colmap.github.io/cameras.html). 
Camera intrinsics calibration is optional but suggested for more accurate and faster camera registration. Just leave the fields with default values if you want to skip this step.
We typically calibrate the camera by capturing a video of a textured plane with really slow camera motion while trying to let target features
cover the full field of view, selecting non-blurry frames, running **COLMAP** on these images.
- **Upload your video**. Short video is more desired. As a reference, our system takes about 37min on a 3s video excluding COLMAP reconstruction part when tested with one NVIDIA GeForce RTX 2080 GPU. 
Our system works on videos of static scenes or dynamic scenes containing moderate motion (consistent motion that is not epipolar-aligned or inconsistent motion (e.g., a waving hand)). Consistent motion (e.g., a moving car) can sometimes be aligned with the epipolar geometry and cause our method, like most others, to estimate the wrong depth. As a rule of thumb, large baseline (camera translation) is preferred especially a baseline that is much larger than scene motion. And it is better when the motion is not parallel to the camera motion.

In [ ]:
# Set camera model and parameters.
camera_model = "SIMPLE_PINHOLE" #@param {type:"string"}
camera_params = "" #@param {type:"string"}

# Upload video.
%mkdir -p data/videos
%pushd data/videos

from google.colab import files
import os
from os.path import join as pjoin

uploaded = files.upload()
assert len(uploaded) == 1
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

video_file = pjoin('data/videos', list(uploaded.keys())[0])
path = pjoin('results', os.path.splitext(os.path.basename(video_file))[0])
print(f"video file: {video_file}, output path: {path}")
print(f"camera model: {camera_model}, camera parameters: {camera_params}")

%popd

/content/consistent_depth/data/videos


TypeError: ignored

## Run

In [ ]:
%%bash -s "$video_file" "$path" "$camera_model" "$camera_params"
# convert python variables to bash variables
video_file="$1"
path="$2"
camera_model="$3"
camera_params="$4"

In [ ]:
!echo "Start!"
!python main.py --video_file "$video_file" --path "$path" \
  --camera_params "$camera_params" --camera_model "$camera_model" \
  --make_video
!echo "Done. Your results are saved at $(pwd)/$path."
!echo "Video results are at $(pwd)/$path/R_hierarchical2_mc/videos/."
!echo "Disparity maps are at $(pwd)/$path/R_hierarchical2_mc/B0.1_R1.0_PL1-0_LR0.0004_BS4_Oadam/depth"